<a href="https://colab.research.google.com/github/rafaeldsouza/mestrado-ReconhecimentoPadrao/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
X = train[list(test.columns)]
y = train[train.columns[~train.columns.isin(test.columns)]]


In [10]:
from sklearn.base import BaseEstimator, TransformerMixin

def extraiPronome(nome):
    return nome.split(',')[1].split('.')[0].strip()

class AtributosDesejados(BaseEstimator, TransformerMixin):
    def __init__(self, excluirName=True):
        self.excluirName = excluirName
    def fit(self, X, y=None):
        self.colunasIndesejadas = ['PassengerId', 'Ticket', 'Cabin']
        if self.excluirName:
            self.colunasIndesejadas.append('Name')
        return self
    def transform(self, X, y=None):
        Xdrop = X.drop(self.colunasIndesejadas,axis=1)
        if 'Name' not in self.colunasIndesejadas:
            Xdrop['Name'] = Xdrop['Name'].apply(extraiPronome)
        return Xdrop


In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas].to_numpy()


In [12]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas].to_numpy()


In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

trataAtributos = Pipeline([
    ('unecaracteristicas', FeatureUnion([
        ('pipenum', Pipeline([
            ('atributos_numericos', AtributosNumericos()),
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])),
        ('pipecat', Pipeline([
            ('atributos_categoricos', AtributosCategoricos()),
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
])


In [91]:
'''
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedKFold
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

pipetotal = Pipeline([
    ('atributosDesejados', AtributosDesejados()),
    ('trataAtributos', trataAtributos),
    ('classificador',RandomForestClassifier())
])

parametros = {
    'atributosDesejados__excluirName': [True, False],
    'classificador__max_depth': [6]
}
modelo = GridSearchCV(pipetotal, param_grid=parametros, 
                         scoring='roc_auc_ovo_weighted',
                         n_jobs=-1,
                         refit=True)

scores = cross_validate(modelo, X, y.values.ravel(),cv=RepeatedKFold())
scores['test_score'], np.mean(scores['test_score']), np.std(scores['test_score'])
'''

(array([0.89692308, 0.83233613, 0.82966961, 0.8874092 , 0.86644385,
        0.88518951, 0.8614418 , 0.86630361, 0.8552757 , 0.87215608,
        0.83893281, 0.85327812, 0.89166776, 0.87929446, 0.87580214,
        0.88846154, 0.86025794, 0.86492154, 0.856621  , 0.88485574,
        0.91607595, 0.88400176, 0.84286115, 0.82133576, 0.87670335,
        0.8960452 , 0.85855263, 0.84252427, 0.88472419, 0.8416028 ,
        0.88139019, 0.92091097, 0.86268939, 0.84869033, 0.85081539,
        0.88057185, 0.8614418 , 0.85339806, 0.88608414, 0.85753664,
        0.86397436, 0.88375734, 0.86385296, 0.90016234, 0.86342305,
        0.84157609, 0.88611887, 0.88419913, 0.86066253, 0.87008929]),
 0.8686602677206748,
 0.021133692737204855)

In [98]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedKFold
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

pipetotal = Pipeline([
    ('atributosDesejados', AtributosDesejados()),
    ('trataAtributos', trataAtributos),
    ('classificador',GradientBoostingClassifier())
])

parametros = {
    'atributosDesejados__excluirName': [True, False],
    'classificador__max_depth': [6]
}
modelo = GridSearchCV(pipetotal, param_grid=parametros, 
                         scoring='roc_auc_ovo_weighted',
                         n_jobs=-1,
                         refit=True)

scores = cross_validate(modelo, X, y.values.ravel(),cv=RepeatedKFold())
scores['test_score'], np.mean(scores['test_score']), np.std(scores['test_score'])

(array([0.84059078, 0.88596491, 0.86830357, 0.87369792, 0.85132289,
        0.83205953, 0.87909483, 0.84585561, 0.89339806, 0.84543689,
        0.89244265, 0.85975212, 0.85875668, 0.84239719, 0.87215608,
        0.81506755, 0.88377445, 0.86130765, 0.86674734, 0.88779199,
        0.87340085, 0.85307443, 0.83086158, 0.90529156, 0.86158755,
        0.88656476, 0.83017152, 0.85314453, 0.86864177, 0.84106495,
        0.86647122, 0.87394068, 0.86483957, 0.87771489, 0.86493808,
        0.84555442, 0.87676482, 0.86343293, 0.83261494, 0.88710317,
        0.88199627, 0.85873016, 0.88643128, 0.85497835, 0.84866275,
        0.83878385, 0.86382924, 0.88153504, 0.85251142, 0.88567802]),
 0.8633246666203263,
 0.019590947725948275)

In [99]:
modelo.fit(X,y)
y_pred = modelo.predict(test)
result = test[['PassengerId']]
result['Survived'] = y_pred
result.to_csv('submission.csv',index=False)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
